# Web_Scraping_Document_Database

In [21]:
# Dependencies 
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
from pprint import pprint

In [22]:
# Path to chromedriver
!which chromedriver

/usr/local/bin/chromedriver


In [23]:
# Chrome Driver Path
chromiumPath ="/usr/local/bin/chromedriver"

In [24]:
# Set Executable Path & Initialize Chrome Browser
executable_path = {'executable_path': chromiumPath}
browser = Browser('chrome', **executable_path, headless=False)

# Scraper #1
### Visit  NASA mars news site

In [25]:
# url to scrape
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [26]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
mars_news_soup = BeautifulSoup(html, 'html.parser')
# delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [31]:
# Scrape the first article title
title1 = mars_news_soup.find('div', class_='content_title').text
title1

"Media Get a Close-Up of NASA's Mars 2020 Rover"

In [32]:
# Scrape the first article teaser paragraph text
paragraph1 = mars_news_soup.find('div', class_='article_teaser_body').text
paragraph1

"The clean room at NASA's Jet Propulsion Laboratory was open to the media to see NASA's next Mars explorer before it leaves for Florida in preparation for a summertime launch."

# Scraper #2
### JPL Mars Space Featured Image 

In [35]:
# URL to scrap
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [36]:
# find and click 'FULL IMAGE'
browser.click_link_by_partial_text('FULL IMAGE')

In [37]:
# Go to 'more info'
browser.click_link_by_partial_text('more info')

In [38]:
#  Parse the resulting html with soup
html = browser.html
image_soup = BeautifulSoup(html, 'html.parser')

In [39]:
# find the relative image  URL
img_url = image_soup.find('figure', class_='lede').a['href']
img_full_url = f'https://www.jpl.nasa.gov{img_url}'
img_full_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16613_hires.jpg'

# Scraper #3
### Mars Weather

In [40]:
# URL to scraped
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [41]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
weath_soup = BeautifulSoup(html, 'html.parser')

In [42]:
# Scrape the tweet info Mars Weather`
mars_weather_tweet = weath_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})

In [43]:
# search within the tweet for the p tag containing the tweet text
mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
mars_weather

'InSight sol 395 (2020-01-06) low -99.7ºC (-147.4ºF) high -17.8ºC (-0.1ºF)\nwinds from the SE at 6.2 m/s (13.8 mph) gusting to 20.3 m/s (45.3 mph)\npressure at 6.40 hPapic.twitter.com/GFhr7zuVp2'

# Scraper #4
### Mars Facts

In [44]:
# Scrape Mars table Facts
url = 'https://space-facts.com/mars/'
table = pd.read_html(url)
df = table[0]
df.columns = ['Property', 'Value']
df

,Property,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [45]:
# Convert to HTML table string
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Property</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC<

# Scraper #5
### Mars Hemispheres

In [46]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [48]:
# load list with links for the hemispheres
hemis_strings = []
links = browser.find_by_css("a.product-item h3")


for hemis in links:
    hemis_strings.append(hemis.text)
    
hemis_strings

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [175]:
hemisphere_image_urls=[]
# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # W find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to our list
    hemisphere_image_urls.append(hemisphere)
    
    #  navigate backwards
    browser.back()

In [174]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]